# misc

## option streak days

In [ ]:
import warnings
import pandas as pd

warnings.simplefilter(action="ignore")

# * copy table, must be sorted by TrainingDateTime
_df = (
    df_training.sort_values("TrainingDateTime")
    .reset_index()[["Id", "TrainingDateTime", "Streak_days"]]
    .copy()
)
# * ensure index starts at 1
_df.index += 1

# * convert to proper datetime
_df["training_date"] = pd.to_datetime(_df.TrainingDateTime).dt.date.astype("datetime64")

# * add gap in days between training of two consecutive records
_df["gap"] = (
    (_df["training_date"] - _df.shift(1)["training_date"])
    # .dt.days                              # convert to number
    # .astype('Int16')                      # use Int type to handle nulls
    # .fillna(dt.timedelta(days=9999))  # fill nulls to include first item
)

# * filter records with gap > 1 to only show streak starts
_df2 = _df[(_df["gap"]).dt.days > 1]

# * use date from previous record as start date
_df2["date_start"] = _df2.shift(1)["training_date"]

# * leap back in time before the gap
_df2["date_end"] = _df2["training_date"] - _df2["gap"]

# * calculate streak as date difference
_df2["streak"] = ((_df2["date_end"] - _df2["date_start"]).dt.days + 1).astype("Int16")

_df2.sort_values(by='streak', ascending=False)